In [1]:
%defaultDatasource jdbc:h2:mem:db

### Normalização
De acordo com a tabela principal, pensamos em modificações e otimizações reconhecendo as dependências funcionais entre as colunas. Percebemos que o país e o ano determinam o PIB anual, PIB per capita e a população. Analogamente, a idade determina a geração de um grupo de pessoas.
Logo, construímos as seguintes tabelas de acordo com a normalização feita:
 
Principal(**<ins>country</ins>**, **<ins>year</ins>**, **<ins>age</ins>**, **<ins>sex</ins>**, suicide_rate);
 
Pais(**<ins>country</ins>**, **<ins>year</ins>**, gdp_for_year, gdp_per_capita, population);
 
Geração(**<ins>age</ins>**, generation).
 
No entanto, a tabela tirada diretamente do Kaggle representa um modelo que não podemos usar diretamente para o nosso banco de dados relacional, logo é necessário a limpeza desses dados. A exemplo, temos a tabela de geração na qual grupos de idades determinam gerações diferentes.
 
Isso não foi feito durante esta etapa do trabalho mas estará feita até a próxima. Por isso, tivemos que considerar a criação excessiva de VIEWS para responder as perguntas que fizemos.
 
Os resultados das execuções das queries encontram-se abaixo.


# Importando Tabelas

In [2]:
DROP TABLE IF EXISTS Principal;
DROP TABLE IF EXISTS Geracao;
DROP TABLE IF EXISTS Pais;

CREATE TABLE  Principal(
    country VARCHAR(200),
    year INTEGER,
    sex VARCHAR(20),
    age VARCHAR(50),
    suicidio INTEGER,
    taxa_suicidio DOUBLE
) AS SELECT
    country,
    year,
    sex,
    age,
    suicides_no,
    suicides_rate
FROM CSVREAD('master.csv');

CREATE TABLE  Geracao(
    age VARCHAR(20),
    generation VARCHAR(20),
) AS SELECT
    age,
    generation
FROM CSVREAD('master.csv');

CREATE TABLE  Pais(
    country VARCHAR(50),
    year INTEGER,
    gdp_for_year BIGINT,
    gdp_per_capita  INTEGER,
    population INTEGER
) AS SELECT
    country,
    year,
    gdp_for_year,
    gdp_per_capita,
    population
FROM CSVREAD('master.csv');


SELECT * FROM Principal;
SELECT * FROM Geracao;
SELECT * FROM Pais;

# Queries
1- Qual país teve maior taxa de suicídio em 2010?

Primeiro criamos uma view que agrupa as maiores de taxa de suicidio com o país no ano de 2010, em seguinte uma view que verifica a maior taxa de suícidio entre esses paises, assim a resposta é encontrada slecionar qual país possui essa taxa, neste caso é a Coreia do Sul.

In [3]:
DROP VIEW IF EXISTS MaiorTaxa;
DROP VIEW IF EXISTS MaiorTaxaPais;

CREATE VIEW MaiorTaxa AS 
SELECT country, MAX(taxa_suicidio) maior
FROM Principal
WHERE year = 2010
GROUP BY country;

CREATE VIEW MaiorTaxaPais AS 
SELECT MAX(maior) maior_pais
FROM MaiorTaxa;

SELECT MT.country
FROM MaiorTaxa MT, MaiorTaxaPais MTP
WHERE MT.maior = MTP.maior_pais;


Republic of Korea

2 - Qual sexo tem a maior taxa de sucidios entre 25 e 34 anos em 2010?

Foi criado uma view que selecionna a taxa de suicídio de ambos os sexos entre 25 e 34 anos em 2010, após isso selecionado o maior, obtendo sexo masculino como resposta.

In [4]:
DROP VIEW ST IF EXISTS;

CREATE VIEW ST AS SELECT sex, MAX(taxa_suicidio) AS taxa
        FROM Principal
        WHERE age = '25-34 years' AND year = 2010
        GROUP BY sex;

SELECT ST.sex 
    FROM (SELECT MAX(taxa) maxima FROM ST) AS MAIOR, ST
    WHERE ST.taxa = MAIOR.maxima;

male

3 - Qual o PIB per capita do país com mais suicídios entre pessoas de 25 e 34 anos?

Foi criado vários views para limpar os dados da tabela e facilitar a busca, a primeira seleciona país, ano e taxa de suicidio entre pessoas de 25 a 34 anos, aí verficamos qual é a maior taxa e em que ano foi, e assim utilizamos a tabela Pais para descbobir o PIB per capita desse país e ano, resultando em 2853$.

In [ ]:
DROP VIEW IF EXISTS MaxSr;
DROP VIEW IF EXISTS MaxTaxa;
DROP VIEW IF EXISTS MaxPais;
DROP VIEW IF EXISTS MaxAno;
DROP VIEW IF EXISTS PaisAno;




CREATE VIEW MaxSr AS 
SELECT country,year,MAX(taxa_suicidio) maior
FROM Principal
WHERE age = '25-34 years'
GROUP BY country,year;

CREATE VIEW MaxTaxa AS 
SELECT year,MAX(maior) maior_pais
FROM MaxSr
Group by year;

CREATE VIEW MaxPais AS 
SELECT country,MAX(maior) maior_pais
FROM MaxSr
Group by country;

CREATE VIEW MaxAno AS 
SELECT MAX(maior) maior
FROM MaxSr;


CREATE VIEW PaisAno AS 
SELECT MP.country, MT.year
FROM MaxPais MP, MaxTaxa MT , MaxAno MA 
WHERE MP.maior_pais = MT.maior_pais AND  MP.maior_pais = MA.maior;

SELECT TOP 1 P.gdp_per_capita
FROM Pais P, PaisAno PA
WHERE P.country = PA.country AND P.year = PA.year
ORDER BY P.gdp_per_capita;


4 - Qual o PIB per capita do país com a menor taxa de suicídio entre os mais idosos(75+ anos)?


Semelhante a 3 usamos views para fazer uma limpeza nos dados, criamos uma  view com a  taxa de suicídio entre os idosos, pais e ano, verificamos a menor taxa de suicídio e com qual país e ano ela está relacionada, assim é possível selecionar qual o PIB desse país nesse ano, resultando em 8347$.

In [ ]:
DROP VIEW PaisAno IF EXISTS;
DROP VIEW AnoEMinTaxa IF EXISTS;
DROP VIEW PaisEMinTaxa IF EXISTS;
DROP VIEW MinTaxa IF EXISTS;
DROP VIEW RecorteIdososMinTaxa IF EXISTS;

CREATE VIEW RecorteIdososMinTaxa AS SELECT country, year, age, MIN(taxa_suicidio) recIdosMinTaxa
    FROM Principal
    WHERE age = '75+ years' AND taxa_suicidio > 0
    GROUP BY country, year;

-- SELECT * FROM RecorteIdososMinTaxa;

CREATE VIEW AnoEMinTaxa AS
    SELECT year, MIN(recIdosMinTaxa) yMinTaxa
        FROM RecorteIdososMinTaxa
        GROUP BY year;

-- SELECT * FROM AnoEMinTaxa;

CREATE VIEW PaisEMinTaxa AS
    SELECT country, MIN(recIdosMinTaxa) cMinTaxa
        FROM RecorteIdososMInTaxa
        GROUP BY country;

-- SELECT * FROM PaisEMinTaxa;

CREATE VIEW MinTaxa AS
    SELECT MIN(recIdosMinTaxa) minTaxa
        FROM RecorteIdososMinTaxa;

-- SELECT * FROM MinTaxa;

CREATE VIEW PaisAno AS
    SELECT PMT.country, ANT.year
        FROM PaisEMinTaxa PMT, AnoEMinTaxa ANT, MinTaxa MT
        WHERE PMT.cMinTaxa = ANT.yMinTaxa AND PMT.cMinTaxa = MT.minTaxa;

-- SELECT * FROM PaisAno;

SELECT TOP 1 P.gdp_per_capita
    FROM Pais P, PaisAno PA
    WHERE P.country = PA.country AND P.year = PA.year
    GROUP BY P.gdp_per_capita;

DROP VIEW PaisAno IF EXISTS;
DROP VIEW AnoEMinTaxa IF EXISTS;
DROP VIEW PaisEMinTaxa IF EXISTS;
DROP VIEW MinTaxa IF EXISTS;
DROP VIEW RecorteIdososMinTaxa IF EXISTS;


5- Qual a variação, em porcentagem, do índice de suicídios do ano 2010 em relação ao ano 2007 no Brasil?

Foi pego a soma do indice de suicídio de pessoas de 25 a 34 no Brasil em 2007 e em 2010, com isso foi calculado a diferença percentual resultando num aumento de aproximadamente 11.74%.

In [ ]:
DROP VIEW IF EXISTS SuicideNoBr2007;
DROP VIEW IF EXISTS SuicideNoBr2010;

CREATE VIEW SuicideNoBr2007 AS
SELECT CAST(SUM(P.suicidio) AS DOUBLE) suicidio
FROM Principal P
WHERE P.country = 'Brazil' AND P.year = 2007 AND P.age = '25-34 years';

CREATE VIEW SuicideNoBr2010 AS
SELECT CAST(SUM(P.suicidio) AS DOUBLE) suicidio
FROM Principal P
WHERE P.country = 'Brazil' AND P.year = 2010 AND P.age = '25-34 years';

SELECT ( ( (S10.suicidio/S07.suicidio) - 1)*100) 
FROM SuicideNoBr2007 S07, SuicideNoBr2010 S10;